In [1]:
cd ..

/Users/didac.fortuny/Didac/repos/ddc/godatathon_2020


In [2]:
import pandas as pd

from src.utils import read_raw_file, save_dataset

# Settings

In [3]:
DATA_RAW_PATH = "data/raw"
VOLUME_FILE = "gx_volume.csv"
GENERICS_FILE = "gx_num_generics.csv"
PACKAGE_FILE = "gx_package.csv"
PANEL_FILE = "gx_panel.csv"
AREA_FILE = "gx_therapeutic_area.csv"
SUBMISSION_FILE = "submission_template.csv"
DATA_CLEAN_PATH = "data/clean"
OUTPUT_NAME = "dataset_complete.csv"

# Retrieve data

In [4]:
volume = read_raw_file(DATA_RAW_PATH, VOLUME_FILE)
volume.head(2)

,country,brand,volume,month_num,month_name
1,country_1,brand_3,18509088.6,-88,Jul
2,country_1,brand_3,19697508.0,-87,Aug


In [5]:
submission = pd.read_csv(f"{DATA_RAW_PATH}/{SUBMISSION_FILE}")
submission.head(2)

,country,brand,month_num,pred_95_low,prediction,pred_95_high
0,country_1,brand_121,0,NaN,NaN,NaN
1,country_1,brand_121,1,NaN,NaN,NaN


In [6]:
competitors = read_raw_file(DATA_RAW_PATH, GENERICS_FILE)
competitors.head(2)

,country,brand,num_generics
1,country_1,brand_3,3
2,country_1,brand_4,1


In [7]:
packages = read_raw_file(DATA_RAW_PATH, PACKAGE_FILE)
packages.head(2)

,country,brand,presentation
1,country_1,brand_3,PILL
2,country_1,brand_4,PILL


In [8]:
channels = read_raw_file(DATA_RAW_PATH, PANEL_FILE)
channels.head(2)

,country,brand,channel,channel_rate
1,country_1,brand_3,B,1.189704
2,country_1,brand_3,D,98.810296


In [9]:
bodyparts = read_raw_file(DATA_RAW_PATH, AREA_FILE)
bodyparts.head(2)

,brand,therapeutic_area
1,brand_1,Nervous_system
2,brand_2,Respiratory_and_Immuno_inflammatory


# Create base table

## Join given and required datasets

In [10]:
volume = volume.drop(columns=["month_name"])
volume["given"] = True
volume.head(2)

,country,brand,volume,month_num,given
1,country_1,brand_3,18509088.6,-88,True
2,country_1,brand_3,19697508.0,-87,True


In [11]:
submission = submission.drop(columns=["pred_95_low", "prediction", "pred_95_high"])
submission["submission"] = True

In [12]:
dataset = pd.concat([volume, submission]).fillna(False)
dataset = dataset.groupby(["country", "brand", "month_num"], as_index=False).sum()
dataset.head(2)

,country,brand,month_num,given,submission
0,country_1,brand_10,-47,True,False
1,country_1,brand_10,-46,True,False


## Add number of competitors

In [13]:
dataset = dataset.merge(competitors, on=["country", "brand"], how="left").fillna(0)
dataset.head(2)

,country,brand,month_num,given,submission,num_generics
0,country_1,brand_10,-47,True,False,6
1,country_1,brand_10,-46,True,False,6


## Add type of drug

In [14]:
packages = pd.get_dummies(packages, columns=["presentation"], prefix="package")
packages.head(2)

,country,brand,package_CREAM,package_EYE_DROP,package_INHALER,package_INJECTION,package_OTHER,package_PATCH,package_PILL
1,country_1,brand_3,0,0,0,0,0,0,1
2,country_1,brand_4,0,0,0,0,0,0,1


In [15]:
dataset = dataset.merge(packages, on=["country", "brand"], how="left").fillna(0)
dataset.head(2)

,country,brand,month_num,given,submission,num_generics,package_CREAM,package_EYE_DROP,package_INHALER,package_INJECTION,package_OTHER,package_PATCH,package_PILL
0,country_1,brand_10,-47,True,False,6,0,0,0,0,0,0,1
1,country_1,brand_10,-46,True,False,6,0,0,0,0,0,0,1


## Add distribution channel rates

In [16]:
channels = pd.pivot_table(channels, columns=["channel"], index=["country", "brand"]).fillna(0)
channels.columns = [f"{c[0]}_{c[1]}" for c in channels.columns]
channels.head(2)

channel_rate_A  channel_rate_B  channel_rate_C  \
country   brand                                                       
country_1 brand_10              0.0        1.015697             0.0   
          brand_102             0.0        0.109766             0.0   

                     channel_rate_D  
country   brand                      
country_1 brand_10        98.984303  
          brand_102       99.890234

In [17]:
dataset = dataset.merge(channels, on=["country", "brand"], how="left").fillna(0)
dataset.head(2)

,country,brand,month_num,given,submission,num_generics,package_CREAM,package_EYE_DROP,package_INHALER,package_INJECTION,package_OTHER,package_PATCH,package_PILL,channel_rate_A,channel_rate_B,channel_rate_C,channel_rate_D
0,country_1,brand_10,-47,True,False,6,0,0,0,0,0,0,1,0.0,1.015697,0.0,98.984303
1,country_1,brand_10,-46,True,False,6,0,0,0,0,0,0,1,0.0,1.015697,0.0,98.984303


## Add body part

In [18]:
bodyparts = pd.get_dummies(bodyparts, columns=["therapeutic_area"], prefix="bodypart")
bodyparts.head(2)

,brand,bodypart_Anti_infectives,bodypart_Antineoplastic_and_immunology,bodypart_Cardiovascular_Metabolic,bodypart_Dermatology,bodypart_Endocrinology_and_Metabolic_Disease,bodypart_Haematology,bodypart_Muscoskeletal_Rheumatology_and_Osteology,bodypart_Nervous_system,bodypart_Obstetrics_Gynaecology,bodypart_Other,bodypart_Parasitology,bodypart_Respiratory_and_Immuno_inflammatory,bodypart_Sensory_organs,bodypart_Systemic_Hormones
1,brand_1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,brand_2,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [19]:
dataset = dataset.merge(bodyparts, on="brand", how="left").fillna(0)
dataset.head(2)

,country,brand,month_num,given,submission,num_generics,package_CREAM,package_EYE_DROP,package_INHALER,package_INJECTION,...,bodypart_Endocrinology_and_Metabolic_Disease,bodypart_Haematology,bodypart_Muscoskeletal_Rheumatology_and_Osteology,bodypart_Nervous_system,bodypart_Obstetrics_Gynaecology,bodypart_Other,bodypart_Parasitology,bodypart_Respiratory_and_Immuno_inflammatory,bodypart_Sensory_organs,bodypart_Systemic_Hormones
0,country_1,brand_10,-47,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,country_1,brand_10,-46,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


## Transform country and brand into dummies

In [40]:
dataset_dummies = pd.get_dummies(dataset[["country", "brand"]], columns=["country", "brand"], prefix=["", ""])
dataset_dummies.columns = [c[1:] for c in dataset_dummies.columns]
dataset = pd.concat([dataset, dataset_dummies], axis=1)
dataset.head(2)

,country,brand,month_num,given,submission,num_generics,package_CREAM,package_EYE_DROP,package_INHALER,package_INJECTION,...,brand_90,brand_91,brand_92,brand_93,brand_94,brand_95,brand_96,brand_97,brand_98,brand_99
0,country_1,brand_10,-47,True,False,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,country_1,brand_10,-46,True,False,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Reorder columns

In [41]:
first_columns = ["given", "submission", "country", "brand"]
last_columns = [c for c in dataset.columns if c not in first_columns]
dataset = dataset[first_columns + last_columns]
dataset.head(2)

,given,submission,country,brand,month_num,num_generics,package_CREAM,package_EYE_DROP,package_INHALER,package_INJECTION,...,brand_90,brand_91,brand_92,brand_93,brand_94,brand_95,brand_96,brand_97,brand_98,brand_99
0,True,False,country_1,brand_10,-47,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,True,False,country_1,brand_10,-46,6,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save complete dataset

In [23]:
dataset.head(10)

,country,brand,month_num,given,submission,num_generics,package_CREAM,package_EYE_DROP,package_INHALER,package_INJECTION,...,bodypart_Endocrinology_and_Metabolic_Disease,bodypart_Haematology,bodypart_Muscoskeletal_Rheumatology_and_Osteology,bodypart_Nervous_system,bodypart_Obstetrics_Gynaecology,bodypart_Other,bodypart_Parasitology,bodypart_Respiratory_and_Immuno_inflammatory,bodypart_Sensory_organs,bodypart_Systemic_Hormones
0,country_1,brand_10,-47,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,country_1,brand_10,-46,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,country_1,brand_10,-45,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,country_1,brand_10,-44,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
4,country_1,brand_10,-43,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
5,country_1,brand_10,-42,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
6,country_1,brand_10,-41,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
7,country_1,brand_10,-40,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
8,country_1,brand_10,-39,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9,country_1,brand_10,-38,True,False,6,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [24]:
save_dataset(dataset, DATA_CLEAN_PATH, OUTPUT_NAME)